In [2]:
!pip install -q transformers sentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00


In [3]:
import torch
from transformers import XLNetModel, XLNetTokenizer
import re
import os
import requests
from tqdm.auto import tqdm
import numpy as np

In [4]:
tokenizer = XLNetTokenizer.from_pretrained("Rostlab/prot_xlnet", do_lower_case=False)

In [5]:
xlnet_men_len = 512

In [6]:
model = XLNetModel.from_pretrained("Rostlab/prot_xlnet",mem_len=xlnet_men_len)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

In [8]:
sequences_Example = '/content/davis_prot_dictionary.txt'
sequences_Example = [re.sub(r"[UZOBX]", "<unk>", sequence) for sequence in sequences_Example]
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [10]:
with torch.no_grad():
    output = model(input_ids=input_ids,attention_mask=attention_mask,mems=None)
    embedding = output.last_hidden_state
    mems = output.mems

In [11]:
embedding = embedding.cpu().numpy()

In [12]:
#if not isinstance(embedding, np.ndarray):
   #embedding = embedding.numpy()


features = []
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    padded_seq_len = len(attention_mask[seq_num])
    seq_emd = embedding[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
    features.append(seq_emd)

In [13]:
print(features)

[array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407,
        -0.21469194,  0.65919745]], dtype=float32), array([[ 0.9576438 , -0.3334456 ,  0.66364574, ...,  0.61825407

Transformer Encoder https://github.com/rohan-paul/MachineLearning-DeepLearning-Code-for-my-YouTube-Channel/tree/master/NLP/Transformer_From_Scratch

In [20]:
!pip install bertviz transformers -q


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
from transformers import BertModel
#from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "/content/davis_prot_dictionary.txt"

model = BertModel.from_pretrained(model_ckpt)
